In [9]:
#Screen Capture

In [10]:
"""import pyscreenshot as ImageGrab
import time
import os
def data_set():
    # Change this to whatever label/subfolder you want
    # e.g. if capturing digit “9”, run data_set() with images_folder="captured_images/9/"
    images_folder = "captured_images/5/"
    os.makedirs(images_folder, exist_ok=True)
    time.sleep(3)
    print("3")
    time.sleep(2)
    print("2")
    time.sleep(1)
    print("1")
    for i in range(13, 101):
        time.sleep(2.5)
        im = ImageGrab.grab(bbox=(70, 230, 920, 1020))
        print(f"Saved {i}.png")
        im.save(f"{images_folder}{i}.png")
    print("All captures done. You may rest now.")
data_set()"""

'import pyscreenshot as ImageGrab\nimport time\nimport os\ndef data_set():\n    # Change this to whatever label/subfolder you want\n    # e.g. if capturing digit “9”, run data_set() with images_folder="captured_images/9/"\n    images_folder = "captured_images/5/"\n    os.makedirs(images_folder, exist_ok=True)\n    time.sleep(3)\n    print("3")\n    time.sleep(2)\n    print("2")\n    time.sleep(1)\n    print("1")\n    for i in range(13, 101):\n        time.sleep(2.5)\n        im = ImageGrab.grab(bbox=(70, 230, 920, 1020))\n        print(f"Saved {i}.png")\n        im.save(f"{images_folder}{i}.png")\n    print("All captures done. You may rest now.")\ndata_set()'

In [ ]:
import os
import time
import glob

import numpy as np
import pandas as pd
import cv2
import pyscreenshot as ImageGrab
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore

# --- Part 1: Dataset Preparation ---

def build_csv_dataset(capture_root="captured_images", output_csv="dataset.csv"):
    # Prepare CSV header
    header = ["label"] + [f"pixel{i}" for i in range(784)]
    if not os.path.exists(output_csv):
        with open(output_csv, "w", newline="") as f:
            f.write(",".join(header) + "\n")

    # Iterate through each label folder
    for label in range(10):
        pattern = os.path.join(capture_root, str(label), "*.png")
        for img_path in glob.glob(pattern):
            im = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if im is None:
                continue

            # Blur + resize
            im = cv2.GaussianBlur(im, (15, 15), 0)
            im = cv2.resize(im, (28, 28), interpolation=cv2.INTER_AREA)

            # Binarize and flatten
            flat = [(1 if px > 100 else 0) for px in im.flatten()]
            row = [label] + flat

            # Append row to CSV
            with open(output_csv, "a", newline="") as f:
                f.write(",".join(map(str, row)) + "\n")
def load_and_split(csv_path="dataset.csv", test_size=0.2, random_state=42):
    df = pd.read_csv(csv_path)
    df = df.sample(frac=1, random_state=random_state)  # shuffle

    X = df.drop(columns=["label"]).values.astype("float32")
    Y = tf.keras.utils.to_categorical(df["label"].values, num_classes=10)

    split = int((1 - test_size) * len(X))
    train_x, test_x = X[:split], X[split:]
    train_y, test_y = Y[:split], Y[split:]
    return train_x, train_y, test_x, test_y
def train_and_save_model(train_x, train_y, test_x, test_y, model_dir="model_tf"):
    # Reshape to (28,28,1)
    train_x = train_x.reshape(-1, 28, 28, 1)
    test_x = test_x.reshape(-1, 28, 28, 1)

    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        train_x, train_y,
        epochs=10,
        batch_size=32,
        validation_data=(test_x, test_y)
    )

    # Save model
    os.makedirs(model_dir, exist_ok=True)
    model.save(os.path.join(model_dir, "model.keras"))
    return model
if __name__ == "__main__":
    build_csv_dataset()
    tx, ty, vx, vy = load_and_split()
    model = train_and_save_model(tx, ty, vx, vy)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5359 - loss: 1.5833 - val_accuracy: 0.8492 - val_loss: 0.4564
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9087 - loss: 0.3091 - val_accuracy: 0.9233 - val_loss: 0.2661
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9654 - loss: 0.1512 - val_accuracy: 0.9630 - val_loss: 0.1377
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9776 - loss: 0.0686 - val_accuracy: 0.9894 - val_loss: 0.0715
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9967 - loss: 0.0241 - val_accuracy: 0.9894 - val_loss: 0.0507
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9985 - loss: 0.0160 - val_accuracy: 0.9762 - val_loss: 0.0596
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9971 - loss: 0.0157 - val_accuracy: 0.9841 - val_loss: 0.0386
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 0.9894 - val_l

In [5]:
import os
import time
import glob

import numpy as np
import pandas as pd
import cv2
import pyscreenshot as ImageGrab
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore
def realtime_recognizer(model_dir="model_tf", crop_box=(70, 230, 920, 1020)):
    # Load saved TensorFlow model
    model = tf.keras.models.load_model(os.path.join(model_dir, "model.keras"))

    os.makedirs("img", exist_ok=True)
    print("Starting TensorFlow digit recognizer. Press ESC or Enter to exit.")

    while True:
        try:
            # Capture screen region
            img = ImageGrab.grab(bbox=crop_box)
            path = os.path.join("img", "current.png")
            img.save(path)

            # Read and preprocess
            frame = cv2.imread(path)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (15, 15), 0)
            _, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY)
            roi = cv2.resize(thresh, (28, 28), interpolation=cv2.INTER_AREA)
            x = (roi.flatten() > 100).astype("float32").reshape(1, 28, 28, 1)

            # Predict
            probs = model.predict(x)
            digit = np.argmax(probs, axis=1)[0]
            print(f"Prediction: {digit}")

            # Display
            cv2.putText(frame, f"Prediction: {digit}", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.imshow("TF Result", frame)

            key = cv2.waitKey(1)
            if key in [27, 13]:
                print("Exiting.")
                break

            time.sleep(1)
        except Exception as e:
            print("Error:", e)

    cv2.destroyAllWindows()
if __name__ == "__main__":
    realtime_recognizer()

Starting TensorFlow digit recognizer. Press ESC or Enter to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction:

In [1]:
import os
import pandas as pd
import tensorflow as tf

def load_and_split(csv_path="dataset.csv", test_size=0.2, random_state=42):
    df = pd.read_csv(csv_path)
    
    df = df.sample(frac=1, random_state=random_state)  # shuffle

    X = df.drop(columns=["label"]).values.astype("float32")
    Y = tf.keras.utils.to_categorical(df["label"].values, num_classes=10)

    split = int((1 - test_size) * len(X))
    train_x, test_x = X[:split], X[split:]
    train_y, test_y = Y[:split], Y[split:]
    return test_x.reshape(-1, 28, 28, 1), test_y

# --- Load model ---
model = tf.keras.models.load_model("model_tf/model.keras")  # or "model_tf/model.h5"

# --- Load test data ---
test_x, test_y = load_and_split()[0], load_and_split()[1]

# --- Evaluate ---
loss, accuracy = model.evaluate(test_x, test_y, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.9711
